In [9]:
import rasterio
import numpy as np
import geopandas as gpd
import rasterio.features

fp_T9 = r"/home/geoai/geoai_data_test2/NPARK_DATA/data_projects/Green Cover Npark Singapore/results (copy)/forest/T9-2022.tif"
fp_T10 = r"/home/geoai/geoai_data_test2/NPARK_DATA/data_projects/Green Cover Npark Singapore/results/forest/T10-2022.tif"
shp_giu = r"/home/skm/SKM16/Tmp/T10.shp"
fp_out_T10 = r"/home/geoai/geoai_data_test2/NPARK_DATA/data_projects/Green Cover Npark Singapore/results/forest/T10-2022_out.tif"

In [10]:
with rasterio.open(fp_T10) as src:
        meta = src.meta
        height, width = src.height, src.width
        tr = src.transform
        crs_img = src.crs
        maskT10 = src.read()

with rasterio.open(fp_T9) as src:
        maskT9 = src.read()

df = gpd.read_file(shp_giu)
if df.crs.to_string() != crs_img.to_string():
    df = df.to_crs(epsg=str(crs_img.to_epsg()))
shapes = df['geometry']
mask_shp = rasterio.features.rasterize(shapes, out_shape=(height, width), transform=tr)
mask_shp = np.array([mask_shp])
print(maskT9.shape)
maskT9 = maskT9*mask_shp
idx_add = np.where(maskT9 != 0)
maskT10[idx_add] = 1
with rasterio.open(fp_out_T10,'w', **meta) as dst:
    dst.write(maskT10)

(1, 3452, 5377)
